In [1]:
import sys

if '/Users/damianstone/Documents/Code/tennis-project/post-triangulation/' not in sys.path:
  sys.path.append('/Users/damianstone/Documents/Code/tennis-project/post-triangulation/')

In [2]:
import cv2 as cv
import numpy as np
import glob
import json

BASE_PATH = '/Users/damianstone/Documents/Code/tennis-project/post-triangulation/'
IMAGE_BASE_PATH = f'{BASE_PATH}/images/'
INTRINSIC_PARAMS_FILE_NAME = 'intrinsic_params.json'

In [ ]:
# load intrinsic parameters
with open(f'{BASE_PATH}/data/{INTRINSIC_PARAMS_FILE_NAME}', "r") as file:
    camera_data = json.load(file)

camera_data

In [3]:
# ----- Determine image size (assumes all images are the same size) -----
sample_img = cv.imread(img_files_left1[0])
img_size = (sample_img.shape[1], sample_img.shape[0])  # (width, height)

# ----- Stereo calibration function -----
def calibrate_stereo(objpoints, imgpoints_left, imgpoints_right, K_left, D_left, K_right, D_right):
    # We assume intrinsics are fixed from earlier calibration.
    flags = cv.CALIB_FIX_INTRINSIC
    stereo_criteria = (cv.TERM_CRITERIA_MAX_ITER + cv.TERM_CRITERIA_EPS, 100, 1e-5)
    ret, K1, D1, K2, D2, R, T, E, F = cv.stereoCalibrate(
        objpoints,
        imgpoints_left,
        imgpoints_right,
        K_left,
        D_left,
        K_right,
        D_right,
        img_size,
        criteria=stereo_criteria,
        flags=flags
    )
    # Compute projection matrices
    P_left = np.hstack((K1, np.zeros((3, 1))))
    P_right = np.hstack((K2, K2 @ T.reshape(-1, 1)))
    return R, T, P_left, P_right

# ----- Calibrate Stereo Pair 1 -----
R1, T1, P_left1, P_right1 = calibrate_stereo(
    objpoints1,
    imgpoints_left1,
    imgpoints_right1,
    intrinsics['CAM_1_LEFT']['K'],
    intrinsics['CAM_1_LEFT']['D'],
    intrinsics['CAM_2_RIGHT']['K'],
    intrinsics['CAM_2_RIGHT']['D']
)

# ----- Calibrate Stereo Pair 2 -----
R2, T2, P_left2, P_right2 = calibrate_stereo(
    objpoints2,
    imgpoints_left2,
    imgpoints_right2,
    intrinsics['CAM_3_LEFT']['K'],
    intrinsics['CAM_3_LEFT']['D'],
    intrinsics['CAM_4_RIGHT']['K'],
    intrinsics['CAM_4_RIGHT']['D']
)

# ----- Collect results -----
results = {
    "Stereo_Pair_1": {
        "P_left": P_left1,
        "P_right": P_right1,
        "R": R1,
        "T": T1
    },
    "Stereo_Pair_2": {
        "P_left": P_left2,
        "P_right": P_right2,
        "R": R2,
        "T": T2
    }
}

# Print the extrinsic calibration results
print("Extrinsic calibration results:")
for pair, data in results.items():
    print(f"{pair}:")
    print("  P_left =\n", data["P_left"])
    print("  P_right =\n", data["P_right"])
    print("  R =\n", data["R"])
    print("  T =\n", data["T"])

NameError: name 'img_files_left1' is not defined

### Draw epipolar lines to know if rectification is needed

In [ ]:
import cv2 as cv
import numpy as np

def draw_epipolar_lines(img_left, img_right, points_left, points_right, F):
    """ Draw epipolar lines on stereo image pair """
    def draw_lines(img, lines, pts):
        r, c = img.shape[:2]
        img_color = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
        for r, pt in zip(lines, pts):
            color = tuple(np.random.randint(0, 255, 3).tolist())
            x0, y0 = map(int, [0, -r[2]/r[1]])
            x1, y1 = map(int, [c, -(r[2] + r[0]*c)/r[1]])
            img_color = cv.line(img_color, (x0, y0), (x1, y1), color, 1)
            img_color = cv.circle(img_color, tuple(pt), 5, color, -1)
        return img_color

    # Compute epilines for left image
    lines_left = cv.computeCorrespondEpilines(points_right.reshape(-1, 1, 2), 2, F).reshape(-1, 3)
    img_left_with_lines = draw_lines(img_left, lines_left, points_left)

    # Compute epilines for right image
    lines_right = cv.computeCorrespondEpilines(points_left.reshape(-1, 1, 2), 1, F).reshape(-1, 3)
    img_right_with_lines = draw_lines(img_right, lines_right, points_right)

    return img_left_with_lines, img_right_with_lines

# Example usage:
F, _ = cv.findFundamentalMat(imgpoints_left, imgpoints_right, cv.FM_LMEDS)
img_left_with_lines, img_right_with_lines = draw_epipolar_lines(img_left, img_right, imgpoints_left, imgpoints_right, F)

cv.imshow("Left Epipolar Lines", img_left_with_lines)
cv.imshow("Right Epipolar Lines", img_right_with_lines)
cv.waitKey(0)
cv.destroyAllWindows()


### rectification

In [ ]:
R1, R2, P1, P2, Q, roi1, roi2 = cv.stereoRectify(
    K_left, D_left, K_right, D_right, img_size, R, T
)
